### WebdriverIO Langchain tools

### Load local LLM 

In [ ]:
# !pip install -U langchain-openai
# !pip install dotenv

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1",
    temperature=0.5,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

In [ ]:
# from langchain_ollama import ChatOllama

# llm = ChatOllama(
#     base_url="http://localhost:11434",
#     model="deepseek-r1:8b",
#     temperature=0.5,)

### open_browser tool definition 

In [ ]:
from langchain.tools import tool
import requests

SERVER_URL = "http://localhost:3000"

@tool
def open_browser(_=None) -> str:
    """Open Chrome browser using WebdriverIO server."""
    try:
        resp = requests.post(f"{SERVER_URL}/open")
        return resp.text
    except Exception as e:
        return f"Error calling WebdriverIO server: {e}"

@tool
def navigate_to_url(url: str) -> str:
    """Navigate to the given URL using WebdriverIO server."""
    try:
        resp = requests.post(f"{SERVER_URL}/navigate", json={"url": url})
        return resp.text
    except Exception as e:
        return f"Error calling WebdriverIO server: {e}"

@tool
def click_action(element_selector: str) -> str:
    """Click an element in Chrome browser using WebdriverIO server."""
    try:
        resp = requests.post(f"{SERVER_URL}/click", json={"selector": element_selector})
        return resp.text
    except Exception as e:
        return f"Error calling WebdriverIO server: {e}"

@tool
def set_value_action(element_selector: str, value: str) -> str:
    """Set value for an element in Chrome browser using WebdriverIO server."""
    try:
        resp = requests.post(f"{SERVER_URL}/set-value", json={"selector": element_selector, "value": value})
        return resp.text
    except Exception as e:
        return f"Error calling WebdriverIO server: {e}"

@tool
def get_page_source() -> str:
    """Get the HTML source of the current page using WebdriverIO server."""
    try:
        resp = requests.post(f"{SERVER_URL}/source")
        return resp.text
    except Exception as e:
        return f"Error calling WebdriverIO server: {e}"

### Hook tool to local LLM

In [ ]:
from langchain.agents import initialize_agent, AgentType
tools = [open_browser, navigate_to_url, click_action, set_value_action, get_page_source]

agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

scenario = (
    "Scenario:\n"
    "1. Navigate to https://www.saucedemo.com/v1/index.html.\n"
    "2. Login with username standard_user and password secret_sauce.\n"
    "3. Add the first 2 products to the cart.\n"
    "4. Click the cart icon and go to the cart page.\n"
    "5. Verify 2 items are added and visible in the cart."
)

response = agent.run(scenario)
print(response)

In [ ]:
webdriverio_test_prompt = (
    "Generate a WebdriverIO (JavaScript) test script for the following scenario:\n"
    "1. Navigate to https://www.saucedemo.com/v1/index.html\n"
    "2. Login with username standard_user and password secret_sauce\n"
    "3. Add the first 2 products to the cart\n"
    "4. Click the cart icon and go to the cart page\n"
    "5. Verify 2 items are added and visible in the cart."
)

wdio_test_code = llm.invoke(webdriverio_test_prompt)
print(wdio_test_code)
js_code = wdio_test_code.content
with open("saucedemo_test.js", "w", encoding="utf-8") as f:
    f.write(js_code)